In [8]:
%%writefile sqmatrixmul.cu

//Square Matrix Multiplication on GPU using cuda

#include <cstdlib>
#include <cassert>
#include <iostream>

using namespace std;

//kernel function
__global__
void matrixMul(int* a, int* b, int* c, int N){
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x *  blockDim.x + threadIdx.x;

  if(row < N && col < N){
    int temp = 0;

    for(int k = 0; k < N; k++){
      temp += a[row*N + k]*b[k*N + col];
    }
    c[row*N + col] = temp;
  }
}

//host function to verify our result
void verifyMul(int* a, int* b, int* c, int N){

  int temp;
  for(int i = 0; i < N; i++){
    for(int j = 0; j < N; j++){
      temp = 0;
      for(int k = 0; k < N; k++){
        temp += a[i*N + k]*b[k*N + j];
      }
      assert(temp == c[i*N + j]);
    }
  }
}

//CUDA does not allow run-time allocation of a 2D matrix, this essentiates the need for linearization of 2D Matrix
void initMatrix(int* a, int N){
  for(int i = 0; i < N*N; i++){
    a[i] = rand() % 100; // number from 0-99
  }
}

int main(){
  int N = 1 << 10;
  size_t sz = N*N*sizeof(int);

  //allocating and initializing matrices
  int *h_a, *h_b, *h_c;
  h_a = (int*)malloc(sz);
  h_b = (int*)malloc(sz);
  h_c = (int*)malloc(sz);

  int *d_a, *d_b, *d_c;
  cudaMalloc(&d_a, sz);
  cudaMalloc(&d_b, sz);
  cudaMalloc(&d_c, sz);


  initMatrix(h_a, N);
  initMatrix(h_b, N);

  cudaMemcpy(d_a, h_a, sz, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b, sz, cudaMemcpyHostToDevice);


  int threads = 16;
  int blocks = (N + threads - 1) / threads;
  dim3 grid_size(blocks, blocks);
  dim3 block_size(threads, threads);

  //Launching Kernel
  matrixMul<<<grid_size, block_size>>>(d_a, d_b, d_c, N);
  cudaDeviceSynchronize();
  cudaMemcpy(h_c, d_c, sz, cudaMemcpyDeviceToHost);

  //Verifying the result
  verifyMul(h_a, h_b, h_c, N);

  return 0;
}



Overwriting sqmatrixmul.cu


In [9]:
!nvcc -o sqmatrixmul sqmatrixmul.cu

In [ ]:
!./sqmatrixmul

In [ ]:
!nvprof --print-gpu-trace ./sqmatrixmul